In [52]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import json
from datetime import datetime
import dateutil
import numpy as np


countries ={
    "languages": "https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory",
    "population":'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
    }

data = requests.get(countries['languages']).text
soup = BeautifulSoup(data,'html.parser')

# find tables on the wiki page
for table in soup.find_all('table'):
    print(table.get('class'))

tables = soup.find_all('table')
table = soup.find('table', class_='wikitable sortable')


df = pd.DataFrame(columns=['Country/Region', 'Official Language', 'Regional Language', 'Minority language', 'National language', 'Widely spoken'])

for row in table.tbody.find_all('tr'):
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        country = columns[0].text.strip()
        off_lang = columns[1].text.strip()
        reg_lang = columns[2].text.strip()
        min_lang = columns[3].text.strip()
        nat_lang = columns[4].text.strip()
        ws_lang = columns[-1].text.strip()

        df = df.append({'Country/Region':country, 'Official Language':off_lang, 'Regional Language': reg_lang, 'Minority language':min_lang, 'National language':nat_lang, 'Widely spoken':ws_lang}, ignore_index=True)



old_names = ['Country/Region', 'Official Language', 'Regional Language', 'Minority language', 'National language', 'Widely spoken']
new_names = ['country', 'official_lang', 'regional_lang', 'minority_lang', 'national_lang', 'widely_spoken_lang']

rename_dict = {name[0]:name[1] for name in zip(old_names,new_names)}

df = df.rename(columns=rename_dict)

def country_cleaning(country_name):
    country_name = re.sub(r'\[[^]]*\]', '', country_name)
    return re.sub(r'([^a-zA-Z])', ' ', country_name)

df['country'] = df.country.map(country_cleaning)

def lang_clean(langs):
    langs_list_output = []
    langs = re.sub(r'\([^)]*\)', '', langs)
    langs = re.sub(r'\[[^]]*\]', '', langs)
    langs_list = langs.split("\n")
    for item in langs_list:
        item = item.strip()
        langs_list_output.append(str(item))
    if '' in langs_list_output:
        return np.nan
    else:
        return list(langs_list_output)

lang_columns = ['official_lang', 'regional_lang', 'minority_lang', 'national_lang', 'widely_spoken_lang']

for item in lang_columns:
    df[item] = df[item].map(lang_clean)

df.set_index('country', inplace=True)

df.sample(20)

['wikitable', 'sortable']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']
['nowraplinks', 'mw-collapsible', 'expanded', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']
['nowraplinks', 'mw-collapsible', 'expanded', 'navbox-subgroup']
['nowraplinks', 'navbox-subgroup']


,official_lang,regional_lang,minority_lang,national_lang,widely_spoken_lang
country,,,,,
Switzerland,"[French, German, Italian, Romansh]",NaN,NaN,NaN,NaN
Transnistria,"[Moldovan, Russian, Ukrainian]",NaN,NaN,NaN,NaN
Cocos Keeling Islands,"[English, Cocos Malay]",NaN,NaN,NaN,[Malay]
South Africa,"[Afrikaans, English, Southern Ndebele, Sotho, ...",NaN,NaN,NaN,NaN
Qatar,[Arabic],NaN,NaN,NaN,NaN
Croatia,[Croatian],"[Serbian, Hungarian, Czech, Slovak, Pannonian ...",NaN,"[Italian, Romani, Slovene]",NaN
Morocco,"[Arabic, Tamazight]",NaN,NaN,[Arabic],[Darija]
Saint Lucia,[English],NaN,NaN,NaN,NaN
Dominica,[English],NaN,NaN,NaN,NaN


In [53]:
pop_data = requests.get(countries['population']).text
pop_soup = BeautifulSoup(pop_data,'html.parser')

# find tables on the wiki page
for table in pop_soup.find_all('table'):
    print(table.get('class'))

tables = pop_soup.find_all('table')
table = pop_soup.find('table', class_='wikitable sortable')


pop_df = pd.DataFrame(columns=['Country/Dependency', 'Numbers', "'%' of the World", 'Date', 'Source (official or from the United Nations)', 'Notes'])

for row in table.tbody.find_all('tr'):
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        country = columns[0].text.strip()
        numbers = columns[1].text.strip()
        percent_numbers = columns[2].text.strip()
        date = columns[3].text.strip()
        source = columns[4].text.strip()
        notes = columns[-1].text.strip()

        pop_df = pop_df.append({'Country/Dependency':country, 'Numbers':numbers, "'%' of the World": percent_numbers, 'Date':date, 'Source (official or from the United Nations)':source, 'Notes':notes}, ignore_index=True)



old_names = ['Country/Dependency', 'Numbers', "'%' of the World", 'Date', 'Source (official or from the United Nations)', 'Notes']
new_names = ['country', 'pop', 'pop_world (percent)', 'pop_date', 'source', 'notes']

rename_dict = {name[0]:name[1] for name in zip(old_names,new_names)}

pop_df = pop_df.rename(columns=rename_dict)

pop_df = pop_df.drop(0)

def string_to_int(col):
    return int(col.replace(',', ''))

def remove_percent(col):
    return float(col.replace('%', ''))

def to_date_conv(col):
    return datetime.strptime(col, '%d %b %Y')

pop_df['pop'] = pop_df['pop'].map(string_to_int).astype(int)

pop_df['pop_world (percent)'] = pop_df['pop_world (percent)'].map(remove_percent).astype(float)

pop_df['pop_date'] = pop_df['pop_date'].map(to_date_conv)

pop_df = pop_df[['country', 'pop', 'pop_world (percent)', 'pop_date']]

pop_df.set_index('country', inplace=True)

pop_df.head()

['box-Update', 'plainlinks', 'metadata', 'ambox', 'ambox-content', 'ambox-Update']
['wikitable', 'sortable']
['nowraplinks', 'hlist', 'mw-collapsible', 'autocollapse', 'navbox-inner']


,pop,pop_world (percent),pop_date
country,,,
China,1411750000,17.60,2022-12-31
India,1375586000,17.20,2022-03-01
United States,334426100,4.17,2023-02-25
Indonesia,275773800,3.44,2022-07-01
Pakistan,235825000,2.94,2022-07-01


In [54]:
combined_df = pop_df.join(df, on='country', how='inner')

combined_df.to_csv('wiki_country_pop_lang.csv', header=True)